In [1]:
#KoBERT
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer
import torch
from kobert.pytorch_kobert import get_pytorch_kobert_model
bertmodel, vocab  = get_pytorch_kobert_model()
tok_path = get_tokenizer()
sp  = SentencepieceTokenizer(tok_path)

using cached model
using cached model
using cached model


In [2]:
# install
#!pip install pytorch-pretrained-bert pytorch-nlp

# BERT imports
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss, NLLLoss
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
#% matplotlib inline

gpu = False
# specify GPU device
if gpu:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

Using TensorFlow backend.


'GeForce GTX 1060 3GB'

In [3]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_labels=17,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.num_labels=num_labels
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
        self.classifier = nn.Linear(hidden_size , num_labels)
        
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        pooled_output = outputs[-1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [4]:
model = BERTClassifier(bertmodel, num_labels=17, dr_rate=0.1).to(device)
model.load_state_dict(torch.load('./modelsave/nonattentionmask100epoch.pt'))

<All keys matched successfully>

In [5]:
def lineToinput_id(line):
    token_sent = sp(line)
    token_sent.insert(0,'[CLS]')
    token_sent.append('[SEP]')
    input_ids = []
    input_ids.append(vocab(token_sent))
    #Set the maximum sequence length. 
    MAX_LEN = 64
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    print(type(input_ids))
    return input_ids

In [6]:
def intent_classifier(line):
    
    input_data = torch.tensor(lineToinput_id(line))
    if gpu:
        input_data = input_data.to(device)
    validation_data = TensorDataset()
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=1)

    
    # Add batch to GPU
    #batch = tuple(t.to(device) for t in batch)
    model.eval()
    #b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(input_data, token_type_ids=None, attention_mask=None)    
    # Move logits and labels to CPU
    #logits = logits[0].detach().cpu().numpy()
    '''
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(input_ids, token_type_ids=None)
    logits = logits[0].detach().cpu().numpy()
        '''
    prob = torch.exp(torch.nn.functional.log_softmax(logits[0],dim=1))
    return (np.argmax(prob),prob.numpy()[0,int(np.argmax(prob))])


    #return logits

In [7]:
def getIntentCode(line):
    
    index,prob = intent_classifier(line)
    
    print(index, prob)
    if prob > 0.95 :
    #do code mapping
        if index == 7:
            return 302

        elif index == 8:
            return 303
        
        else:
            return -1

    else:
        return -1



In [29]:
code = getIntentCode("A 씨 돌려봐")
print(code)

<class 'numpy.ndarray'>
tensor(7) 0.9817301
302


In [9]:
index,prob = intent_classifier(line)
        



NameError: name 'line' is not defined